In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# !pip install import-ipynb

In [ ]:
# !cp '/content/drive/MyDrive/Colab Notebooks/Covid19/Covid19_CrAsppNet.ipynb' .
# !cp '/content/drive/MyDrive/Colab Notebooks/Covid19/Covid19_CRNet.ipynb' .
# !cp '/content/drive/MyDrive/Colab Notebooks/Covid19/Covid19_CrAsppNet.ipynb' .

In [ ]:
# %cd Desktop/Minh\ Hieu

In [ ]:
import torch
import torch.nn as nn
import import_ipynb
import torchvision.transforms.functional as TF
from collections import OrderedDict
from Covid19_CrAsppNet import CRASPPNet

In [ ]:
class Conv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(Conv, self).__init__()
    self.layers = OrderedDict([
                               ('conv_1', nn.Conv2d(
                                   in_channels = in_channels,
                                   out_channels = out_channels,
                                   kernel_size = 7, 
                                   stride = 1,
                                   padding = 3,
                                   bias=False
                               )),
                               ('norm_1', nn.BatchNorm2d(out_channels)),
                               ('relu_1', nn.ReLU(inplace=True)),     
                              #  ('conv_2', nn.Conv2d(
                              #      in_channels = out_channels,
                              #      out_channels = out_channels,
                              #      kernel_size = 7, 
                              #      stride = 1,
                              #      padding = 3,
                              #      bias=False
                              #  )),
                              #  ('norm_2', nn.BatchNorm2d(out_channels)),
                              #  ('relu_2', nn.ReLU(inplace=True)),         
    ])
    self.conv = nn.Sequential(self.layers)
  
  def forward(self, x):
    return self.conv(x)

In [ ]:
class ASPPWrapper(nn.Sequential):
  def __init__(self, aspp_module, in_channels=256, out_channels=128):
    super(ASPPWrapper, self).__init__(
        aspp_module,
        Conv(in_channels=in_channels, out_channels=out_channels)
    )

In [ ]:
class UpTransConv(nn.Module):
  def __init__(self, in_channels, out_channels, size=None, scale_factor=None):
    super(UpTransConv, self).__init__()
    self.layers = OrderedDict([
                               ('up_sample', nn.Upsample(
                                   size=size,
                                   scale_factor=scale_factor,
                                   mode='bilinear',
                                   align_corners=False
                               )),       
                              #  ('trans_conv', nn.Conv2d(
                              #      in_channels = in_channels,
                              #      out_channels = out_channels,
                              #      kernel_size = 7, 
                              #      stride = 1,
                              #      padding = 3,
                              #      bias=True
                              #  ))        
                               ('trans_conv', nn.ConvTranspose2d(
                                  in_channels=in_channels,
                                  out_channels=out_channels,
                                  kernel_size=7,
                                  stride=1,
                                  padding=3,
                                  bias=True
                               ))
    ])
    self.conv = nn.Sequential(self.layers)
  def forward(self, x):
    return self.conv(x)

In [ ]:
class CustomFinalCRASPPReUNet(CRASPPNet):
  def __init__(self, img_dimwh=224, in_channels=3, out_channels=1, downs_features=[32, 64, 128], atrous_rates=[6, 12, 18], scale_factor=3):
    super(CustomFinalCRASPPReUNet, self).__init__(img_dimwh=img_dimwh, in_channels=in_channels, downs_features=downs_features, atrous_rates=atrous_rates)
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
    self.downs = nn.ModuleDict({})
    self.aspp_wrapper = ASPPWrapper(
        self.aspp,
        in_channels=downs_features[-1] * 2,
        out_channels=downs_features[-1]
    )

    for i, feature in enumerate(downs_features):
      self.downs.update({f"down_{i}" : Conv(in_channels=in_channels, out_channels=feature)})
      in_channels = feature

    expand_features = downs_features[0] * scale_factor * scale_factor 
    self.main_up = nn.Sequential(
        Conv(
            in_channels=downs_features[-1],
            out_channels=expand_features
        ),
        UpTransConv(
            in_channels=expand_features,
            out_channels=int(expand_features/scale_factor),
            scale_factor=scale_factor
        )
    )

    self.small_up = UpTransConv(
        in_channels=downs_features[-1],
        out_channels=downs_features[-2],
        size=self._cal_dim_down__(img_dimwh, len(downs_features)) * scale_factor
    )
    self.main_small_conv = Conv(
        in_channels=downs_features[-2]+int(expand_features/scale_factor),
        out_channels=int(expand_features/scale_factor)
    )

    segmentation_expand_features = int((img_dimwh / (self._cal_dim_down__(img_dimwh, len(downs_features)) * scale_factor)) * downs_features[0])
    self.small_refine = Conv(in_channels=downs_features[1], out_channels=downs_features[0])
    self.segment_branch = nn.ModuleDict({
        "middle_0" : Conv(
            in_channels=int(expand_features/scale_factor),
            out_channels=int(expand_features/scale_factor/2)
            ),
        "middle_1" : Conv(
            in_channels=downs_features[0]+int(expand_features/scale_factor/2),
            out_channels=int(expand_features/scale_factor/2)
            ),
        "middle_2" : Conv(
            in_channels=int(expand_features/scale_factor/2),
            out_channels=segmentation_expand_features + 1
            ),
        "up" : UpTransConv(
            in_channels=segmentation_expand_features + 1,
            out_channels=downs_features[0],
            size=img_dimwh
        ),
        "top_1" : Conv(
            in_channels=downs_features[0]*2,
            out_channels=downs_features[0]
            ),
        "top_2" : nn.Conv2d(
            in_channels=downs_features[0],
            out_channels=out_channels,
            kernel_size=1
        )
    })

    self.tripple_pool = nn.MaxPool2d(
        kernel_size=7,
        stride=3,
        padding=3 
    )
    self.classification_branch = nn.ModuleDict({
        "middle" : Conv(
            in_channels=int(expand_features/scale_factor)+int(expand_features/scale_factor/2),
            out_channels=downs_features[-1]
        ),
         "bottom" : Conv(
            in_channels=downs_features[-1]*2,
            out_channels=downs_features[-1]
        )
    })
    self.bottom.linear=nn.Linear(
        in_features=(int(self._cal_dim_down__(img_dimwh, len(downs_features))/2)) ** 2 * downs_features[-1],
        out_features=2
    )

  def _cal_dim_down__(self, img_dimwh, index):
    globavgpool_dimwh = img_dimwh
    for i in range(index):
      globavgpool_dimwh = int(globavgpool_dimwh / 2 - 0.5)
    return globavgpool_dimwh

  def forward(self, x):
    copy_dict = {}

    ### Down
    x = self.downs["down_0"](x)
    copy_dict.update({"down_0" : x})
    x = self.pool(x)
    x = self.downs["down_1"](x)
    copy_dict.update({"down_1" : x})

    ### Continue Down
    x = self.pool(x)
    x = self.downs["down_2"](x)
    copy_dict.update({"down_2" : x})

    ### Continue Down
    x = self.pool(x)

    ### Bottom
    x = self.aspp_wrapper(x)
    copy_dict.update({"aspp" : x})

    ### Up
    x = self.main_up(x)

    ### Middle Up
    x_small_up = self.small_up(copy_dict["down_2"])
    x = torch.cat((x_small_up, x), dim=1)
    x = self.main_small_conv(x)
    copy_dict.update({"main_small_up_conv" : x})

    ### Segmentation
    x = self.segment_branch["middle_0"](x)
    x_same_level = TF.resize(copy_dict["down_1"], size=x.shape[2:])
    x_same_level = self.small_refine(x_same_level)
    x = torch.cat((x_same_level, x), dim=1)
    x = self.segment_branch["middle_1"](x)
    copy_dict.update({"segmentation_low_level" : x})
    x = self.segment_branch["middle_2"](x)
    x = self.segment_branch["up"](x)
    x = torch.cat((copy_dict["down_0"], x), dim=1)
    x = self.segment_branch["top_1"](x)
    x_segmentation = self.segment_branch["top_2"](x)

    ### Classification
    x = torch.cat((copy_dict["main_small_up_conv"], copy_dict["segmentation_low_level"]), dim=1)
    x = self.classification_branch["middle"](x)
    x = self.tripple_pool(x)
    x = torch.cat((copy_dict["aspp"], x), dim=1)
    x = self.classification_branch["bottom"](x)
    x = self.bottom['globavgpool'](x)
    x = torch.flatten(x, 1)
    x_classification = self.bottom['linear'](x)

    return x_classification, x_segmentation

In [ ]:
def test_final_crasppreu():
  x = torch.randn((16, 3, 224, 224)).cuda()
  model = CustomFinalCRASPPReUNet(img_dimwh=224).cuda()
  cls, seg = model(x)
  print(cls.shape),
  print(seg.shape)

In [ ]:
if __name__ == '__main__':
  test_final_crasppreu()